# issue
* tokenize 작업의 속도가 굉장히 느림 개선방법이 있다면 고민해봐야함 (작업내용 1차적으로 보존)
* 2만개까지는 heap meomory 에러가 발생하지 않았음 따라서 2만개 이상일때는 분할작업진행

In [1]:
# import
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm,tqdm_notebook  # 진행과정 시각화
tqdm.pandas() #apply사용
from datetime import timedelta  # 시간날짜
from apyori import apriori  # 연관분석

import re
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import gc
import os
import math 

import requests
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup  # 크롤링

import wordcloud
from PIL import Image
import random

import konlpy
from konlpy.tag import Komoran,Okt,Kkma,Twitter  # 자연어처리
# from eunjeon import Mecab #은전한닢
okt = Okt(max_heap_size=5120)
# mecab = Mecab()
komoran = Komoran(userdic='user_dictionary.txt')

#한글깨짐방지
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

from IPython.core.display import display, HTML
display(HTML('<style>.container {width:100% !important; }</style>'))

C:\Users\User\Anaconda3\lib\site-packages\jpype\_core.py:217: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
def tokenized(dataframe, keyword, stopword, batch_size = 15000):
    '''
    okt : 형태소분석기 무엇사용?
    dataframe : 분석할 df
    keyword : sw에 추가하기 위함
    '''

    # stopword에 검색어 추가
    stopword.append(keyword.split(' ')[0])
    
    start = 0
    end = len(dataframe)
    
    print(keyword)
    if end<=batch_size:    
        # tqdm.pandas(desc = '토큰화')
        token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: okt.pos(x, norm=True, stem=True))
        token_df_e = token_df.explode()
        # tqdm.pandas(desc = '단어추출')
        token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
        token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

        # 클리닝
        # tqdm.pandas(desc = '클리닝')
        token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
        token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]
        
        #저장
        main_df = pd.merge(dataframe, token_noun.groupby(level=0).agg(list).rename(
            'Noun'), how='left', left_index=True, right_index=True)
        main_df = pd.merge(main_df, token_adj.groupby(level=0).agg(list).rename(
            'Adjective'), how='left', left_index=True, right_index=True)
        main_df.to_csv(f'./output/token/{keyword}_{start}~{end}.csv')
        
    else : #20000개 이상인경우 분할 작업
        final_end = len(dataframe)
        epoch = math.ceil(final_end/batch_size)
        start = 0
        end = batch_size
        while epoch != 0:
            # tqdm.pandas(desc = '토큰화')
            token_df = dataframe.iloc[start:end]['full_text'].progress_apply(lambda x: okt.pos(x, norm=True, stem=True))
            token_df_e = token_df.explode()
            # tqdm.pandas(desc = '단어추출')
            token_noun = token_df_e.apply(lambda x: x[0] if x[1] == 'Noun' else np.nan).dropna()
            token_adj = token_df_e.apply(lambda x: x[0] if x[1] == 'Adjective' else np.nan).dropna()

            # 클리닝
            # tqdm.pandas(desc = '클리닝')
            token_noun = token_noun[token_noun.apply(lambda word: (len(word) > 1) & (word not in sw))]
            token_adj = token_adj[token_adj.apply(lambda word: (len(word) > 1) & (word not in sw))]

            #저장
            main_df = pd.merge(dataframe.iloc[start:end], token_noun.groupby(level=0).agg(list).rename(
                'Noun'), how='left', left_index=True, right_index=True)
            main_df = pd.merge(main_df, token_adj.groupby(level=0).agg(list).rename(
                'Adjective'), how='left', left_index=True, right_index=True)
            os.makedirs(f'./output/token/{keyword}',exist_ok=True)
            main_df.to_csv(f'./output/token/{keyword}/{keyword}_{start}~{end}.csv')
                   
            #next turn
            if epoch >2 :
                start += batch_size
                end += batch_size
            else :      
                start += batch_size
                end = final_end
            
            epoch -= 1
            gc.collect()

### heap 메모리 부족 오류 발생
* heap메모리를 4GB부여하고, 15000개 단위로 분할하여 작업진행 (2만은 가끔 메모리터짐)
* 저장시 기본사항을 붙일수 있게 작업
* 진행이 길어질수록 속도감소(메모리문제로 추정)하기 때문에 재실행필요

In [3]:
#임시 속초/속초해수욕장
sw = list(pd.read_excel("stopword(cp949).xlsx",encoding = 'cp949')['불용어']) #불용어 불러오기
path = 'D:/Python/블로그크롤링/output/크롤링/통합/'
file_list = os.listdir(path)
for file in tqdm_notebook(file_list):
    file_name = file.split('.')[0]
    main_df = pd.read_csv(path + file)
    keyword = file.split('_')[0]
    tokenized(main_df, keyword, sw)
    gc.collect()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|                                                                                        | 0/15000 [00:00<?, ?it/s]

양양 +강원도


  0%|                                                                                 | 5/5222 [00:00<02:45, 31.46it/s]

오산 +강원도


...

  0%|                                                                                 | 3/5938 [00:00<05:56, 16.66it/s]

하조대해수욕장 +양양


  0%|                                                                                          | 0/574 [00:00<?, ?it/s]

후진항 +양양


100%|████████████████████████████████████████████████████████████████████████████████| 574/574 [00:37<00:00, 15.17it/s]


In [4]:
file_list[33:]

['양양 +강원도_통합_47915.csv',
 '오산 +강원도_통합_5222.csv',
 '오산 +양양_통합_2115.csv',
 '오산해수욕장 +양양_통합_887.csv',
 '인구 +강원도_통합_12606.csv',
 '인구 +양양_통합_5113.csv',
 '인구리 +강원도_통합_490.csv',
 '전진리 +강원도_통합_540.csv',
 '죽도 +양양_통합_8408.csv',
 '하광정리 +강원도_통합_639.csv',
 '하조대 +강원도_통합_11196.csv',
 '하조대 +양양_통합_14005.csv',
 '하조대해수욕장 +양양_통합_5938.csv',
 '후진항 +양양_통합_574.csv']

### check

In [5]:
# file = '광진 +양양_통합_5044.csv'
# file_name = file.split('.')[0]
# main_df = pd.read_csv(path + file)
# keyword = file.split('_')[0]
# tokenized(main_df,file_name,keyword,sw)

In [6]:
file_list

['강릉 +강원도_통합_53832.csv',
 '강원도_통합_61330.csv',
 '경포해수욕장 +강원도_통합_5334.csv',
 '광진 +강원도_통합_740.csv',
 '광진 +양양_통합_472.csv',
 '광진리 +강원도_통합_401.csv',
 '기사문 +강원도_통합_998.csv',
 '기사문 +양양_통합_1773.csv',
 '기사문항 +양양_통합_888.csv',
 '낙산 +강원도_통합_12768.csv',
 '낙산 +양양_통합_15083.csv',
 '낙산항 +양양_통합_386.csv',
 '낙산해수욕장_통합_8187.csv',
 '남애 +강원도_통합_1903.csv',
 '남애 +양양_통합_2127.csv',
 '남애항 +양양_통합_3168.csv',
 '동산 +강원도_통합_7536.csv',
 '동산 +양양_통합_1921.csv',
 '동산항 +양양_통합_476.csv',
 '동호 +강원도_통합_2582.csv',
 '동호 +양양_통합_2548.csv',
 '동호해변 +양양_통합_1911.csv',
 '물치 +강원도_통합_1276.csv',
 '물치 +양양_통합_1909.csv',
 '물치리 +강원도_통합_259.csv',
 '물치항 +양양_통합_2701.csv',
 '설악해수욕장 +강원도_통합_955.csv',
 '설악해수욕장 +양양_통합_1255.csv',
 '속초 +강원도_통합_57240.csv',
 '속초해수욕장 +강원도_통합_22816.csv',
 '수산 +강원도_통합_21766.csv',
 '수산 +양양_통합_4956.csv',
 '수산항 +양양_통합_2638.csv',
 '양양 +강원도_통합_47915.csv',
 '오산 +강원도_통합_5222.csv',
 '오산 +양양_통합_2115.csv',
 '오산해수욕장 +양양_통합_887.csv',
 '인구 +강원도_통합_12606.csv',
 '인구 +양양_통합_5113.csv',
 '인구리 +강원도_통합_490.csv',
 '전진리 +강원도_통합_540.csv',
 '죽도 +양양